### *Import Libraries*

In [1]:
import pandas as pd
import numpy as np

from datetime import datetime as dt

### *Data Loading*

In [ ]:
df_tran = pd.read_csv("data\LI-Small_Trans.csv")

In [ ]:
df_acc = pd.read_csv("data\LI-Small_accounts.csv")

In [4]:
df_tran.info()

<class 'pandas.DataFrame'>
RangeIndex: 6924049 entries, 0 to 6924048
Data columns (total 11 columns):
 #   Column              Dtype  
---  ------              -----  
 0   Timestamp           str    
 1   From Bank           int64  
 2   Account             str    
 3   To Bank             int64  
 4   Account.1           str    
 5   Amount Received     float64
 6   Receiving Currency  str    
 7   Amount Paid         float64
 8   Payment Currency    str    
 9   Payment Format      str    
 10  Is Laundering       int64  
dtypes: float64(2), int64(3), str(6)
memory usage: 581.1 MB


In [5]:
df_acc.info()

<class 'pandas.DataFrame'>
RangeIndex: 712688 entries, 0 to 712687
Data columns (total 5 columns):
 #   Column          Non-Null Count   Dtype
---  ------          --------------   -----
 0   Bank Name       712688 non-null  str  
 1   Bank ID         712688 non-null  int64
 2   Account Number  712688 non-null  str  
 3   Entity ID       712688 non-null  str  
 4   Entity Name     712688 non-null  str  
dtypes: int64(1), str(4)
memory usage: 27.2 MB


In [6]:
df_tran['Is Laundering'].sum()

np.int64(3565)

In [7]:
df_acc.head()

,Bank Name,Bank ID,Account Number,Entity ID,Entity Name
0,China Bank #2820,314693,81B86A280,800D8CCF0,Corporation #41344
1,France Bank #4585,311253,8187FEA80,800B505E0,Corporation #54497
2,China Bank #2242,39996,803961E00,800D03F60,Partnership #36904
3,National Bank of Newport,331440,81B075800,801567C10,Corporation #16224
4,UK Bank #33,135417,80CF87C80,801085E00,Partnership #72930


In [8]:
df_tran.head()

,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering
0,2022/09/01 00:08,11,8000ECA90,11,8000ECA90,3195403.00,US Dollar,3195403.00,US Dollar,Reinvestment,0
1,2022/09/01 00:21,3402,80021DAD0,3402,80021DAD0,1858.96,US Dollar,1858.96,US Dollar,Reinvestment,0
2,2022/09/01 00:00,11,8000ECA90,1120,8006AA910,592571.00,US Dollar,592571.00,US Dollar,Cheque,0
3,2022/09/01 00:16,3814,8006AD080,3814,8006AD080,12.32,US Dollar,12.32,US Dollar,Reinvestment,0
4,2022/09/01 00:00,20,8006AD530,20,8006AD530,2941.56,US Dollar,2941.56,US Dollar,Reinvestment,0


### *Normalize feature*

In [9]:
df_tran.rename(columns={'Account': 'from_acc', 
                        'Account.1': 'to_acc',
                        'Amount Received': 'amount_get',
                        'Amount Paid': 'amount_send',
                        'Timestamp': 'timestamp',
                        'Payment Format': 'payment_type',
                        'Receiving Currency': 'to_curr',
                        'Payment Currency': 'from_curr',
                        'From Bank': 'from_bank',
                        'To Bank': 'to_bank',
                        'Is Laundering': 'is_laundering'
                        }, inplace=True)

In [16]:
df_tran['is_self_tx'] = (df_tran['from_acc'] == df_tran['to_acc']).astype(int)
df_tran['is_cross_bank'] = (df_tran['from_bank'] != df_tran['to_bank']).astype(int)
df_tran['timestamp'] = pd.to_datetime(df_tran['timestamp'])
df_tran['hour'] = df_tran['timestamp'].dt.hour
df_tran['log_amount_get'] = np.log1p(df_tran['amount_get'])
df_tran['log_amount_send'] = np.log1p(df_tran['amount_send'])

In [17]:
df_tran.head()

,timestamp,from_bank,from_acc,to_bank,to_acc,amount_get,to_curr,amount_send,from_curr,payment_type,is_laundering,is_self_tx,is_cross_bank,hour,log_amount_get,log_amount_send
0,2022-09-01 00:08:00,11,8000ECA90,11,8000ECA90,3195403.00,US Dollar,3195403.00,US Dollar,Reinvestment,0,1,0,0,14.977224,14.977224
1,2022-09-01 00:21:00,3402,80021DAD0,3402,80021DAD0,1858.96,US Dollar,1858.96,US Dollar,Reinvestment,0,1,0,0,7.528310,7.528310
2,2022-09-01 00:00:00,11,8000ECA90,1120,8006AA910,592571.00,US Dollar,592571.00,US Dollar,Cheque,0,0,1,0,13.292228,13.292228
3,2022-09-01 00:16:00,3814,8006AD080,3814,8006AD080,12.32,US Dollar,12.32,US Dollar,Reinvestment,0,1,0,0,2.589267,2.589267
4,2022-09-01 00:00:00,20,8006AD530,20,8006AD530,2941.56,US Dollar,2941.56,US Dollar,Reinvestment,0,1,0,0,7.987035,7.987035
